In [1]:
import pandas as pd

In [6]:
cost = pd.read_csv("URW_datachallenge/datasets/cost.csv")
catalogue = pd.read_excel("URW_datachallenge/datacatalogueHEC.xlsx")
dwell = pd.read_csv("URW_datachallenge/datasets/dwell_time.csv")
mall = pd.read_csv("URW_datachallenge/datasets/mall.csv")
flow = pd.read_csv("URW_datachallenge/datasets/people_flow.csv")



FileNotFoundError: [Errno 2] No such file or directory: 'URW_datachallenge/datasets/cost.csv'

In [25]:
catalogue.head(20)

,Column Name,Data Type,Data Range,Example Value,Description
0,store_name,string,Free Text,"""Zara""",Store commercial name
1,store_id,int,[0-999999999],"""111""",Unique store identifier
2,mall_id,int,[0-999999999],"""543""",Unique mall identifier
3,store_year_opened,datetime,01/01/1970 - Now,07/05/2021\t,Store opening date
4,store_year_closed,datetime,01/01/1970 - Now,11/02/2021\t,Store closing date
5,store_branch_level_1,string,Free Text,"""Services""",Primary business category
6,store_branch_level_2,string,Free Text,"""Sweet moments""",Secondary business category
7,store_branch_level_3,string,Free Text,"""Monobrand mainstream""",Tertiary business category
8,store_sales_r12m,float,[0-999999999.0],"""450699""",Rolling 12 months sales
9,store_rent_r12m,float,[0-999999999.0],"""546236""",Rolling 12 months rent


We will focus only on the malls that hjave traffic data. As the "Client" said that only those malls are the most important to track and not use the mapping_mal.json file as it has inacuracies

In [106]:
traffic = pd.read_csv("referenciel_traffic.csv")

print("Percentage on NAN values in Traffic dataset")
print(traffic.isna().sum()/ traffic.shape[0] * 100)

print("Before filtering we have ",traffic.shape[0], "rows")
traffic = traffic[traffic.store_id.notna() & traffic.external_mall_id.notna()]
print("After filtering we have ",traffic.shape[0], "rows")
traffic.head()

Percentage on NAN values in Traffic dataset
store_id            52.859004
block_id            87.164025
external_mall_id    87.164025
dtype: float64
Before filtering we have  33001 rows
After filtering we have  4028 rows


,store_id,block_id,external_mall_id
10,258.0,7686.0,13.0
13,294.0,7742.0,13.0
14,717.0,7205.0,19.0
23,1914.0,6990.0,33.0
24,1914.0,6745.0,33.0


In [142]:

# Find duplicate store_id values in the traffic DataFrame
duplicate_store_ids = traffic[traffic.duplicated('store_id', keep=False)]
print("Duplicate store_id values:")
print(duplicate_store_ids)

Duplicate store_id values:
       store_id  block_id  external_mall_id
23       1914.0    6990.0              33.0
24       1914.0    6745.0              33.0
125       340.0    7600.0              13.0
126       340.0    7664.0              13.0
203      1355.0    4742.0              10.0
...         ...       ...               ...
31349    2237.0    6779.0              33.0
32811    1351.0    3538.0              10.0
32812    1351.0    4565.0              10.0
32911    5866.0      60.0               7.0
32912    5866.0    4896.0               7.0

[184 rows x 3 columns]


There are 92 store_id thats have 2 different block_id, wjhat does this mean and how do we merge knowing this ?

In [110]:
stores = pd.read_csv("store.csv")
print(f"We originaly have {stores.shape[0]} stores in our dataset but after filtering for stores that have traffic data we only have")

stores = stores[stores.store_id.isin(traffic.store_id)]

print(stores.shape[0], "stores")
stores.head()

We originaly have 15551 stores in our dataset but after filiterring for stores that have traffic data we only have
3933 stores


,store_name,store_id,mall_id,store_year_opened,store_year_closed,store_branch_level_1,store_branch_level_2,store_branch_level_3,store_sales_r12m,store_rent_r12m,store_area
1,store_1,1,0,28/10/2010,NaN,Services,Clothing-related services,Clothes repairs,1693316.0,663785.0,379.0
2,store_2,2,0,28/10/2010,NaN,Culture & Media & Technology,Electronics and household appliances,Monobrand electronics,2954560.0,326322.0,162.0
3,store_3,3,0,28/10/2010,NaN,Bags & Footwear & Accessories,Accessories,"Mixed: small accessories, bags and fancy jewel...",9706951.0,1189964.0,268.0
4,store_4,4,0,28/10/2010,NaN,Services,Photo services,Photo Development,4217850.0,145534.0,33.0
5,store_5,5,0,23/10/1990,NaN,Health & Beauty,"Perfumery, beautycare",Monobrand mainstream,1645623.0,503069.0,320.0


In [146]:
# merge store and traffic. Now we should have a mappiong between mall_id and external_mall_id
store_traffic = stores.merge(traffic, on='store_id', how='inner')
store_traffic.head()

,store_name,store_id,mall_id,store_year_opened,store_year_closed,store_branch_level_1,store_branch_level_2,store_branch_level_3,store_sales_r12m,store_rent_r12m,store_area,block_id,external_mall_id
0,store_1,1,0,28/10/2010,NaN,Services,Clothing-related services,Clothes repairs,1693316.0,663785.0,379.0,7643.0,13.0
1,store_2,2,0,28/10/2010,NaN,Culture & Media & Technology,Electronics and household appliances,Monobrand electronics,2954560.0,326322.0,162.0,7569.0,13.0
2,store_3,3,0,28/10/2010,NaN,Bags & Footwear & Accessories,Accessories,"Mixed: small accessories, bags and fancy jewel...",9706951.0,1189964.0,268.0,7646.0,13.0
3,store_4,4,0,28/10/2010,NaN,Services,Photo services,Photo Development,4217850.0,145534.0,33.0,7672.0,13.0
4,store_5,5,0,23/10/1990,NaN,Health & Beauty,"Perfumery, beautycare",Monobrand mainstream,1645623.0,503069.0,320.0,7603.0,13.0


In [154]:
flow_dwell= pd.merge(flow, dwell, on=['block_id','external_mall_id'], how='inner')
flow_dwell.head()

,block_id,external_mall_id,store_visits_monthly_average,store_window_flow_monthly_average,store_visits_daily_average,store_window_flow_daily_average,store_total_visits,store_total_window_flow,days_with_traffic_data,average_dwell_time,median_dwell_time
0,2821,12,13296.69,561316.62,473.58,19992.10,172857,7297116,365,436.743706,240.0
1,7750,13,12707.89,600629.56,457.48,21622.66,114371,5405666,250,2826.522727,2820.0
2,4482,10,4316.31,210206.85,153.31,7466.36,56112,2732689,366,296.891672,120.0
3,2331,24,4060.69,173517.31,144.23,6163.18,52789,2255725,366,2788.025950,2820.0
4,45,7,44255.50,525790.38,1580.55,18778.23,354044,4206323,224,843.595047,360.0


In [157]:
merged = pd.merge(store_traffic, flow_dwell, on=['block_id','external_mall_id'], how='inner')
merged.head()

,store_name,store_id,mall_id,store_year_opened,store_year_closed,store_branch_level_1,store_branch_level_2,store_branch_level_3,store_sales_r12m,store_rent_r12m,...,external_mall_id,store_visits_monthly_average,store_window_flow_monthly_average,store_visits_daily_average,store_window_flow_daily_average,store_total_visits,store_total_window_flow,days_with_traffic_data,average_dwell_time,median_dwell_time
0,store_1,1,0,28/10/2010,NaN,Services,Clothing-related services,Clothes repairs,1693316.0,663785.0,...,13.0,3048.89,284206.00,111.09,10355.68,27440,2557854,247,389.212464,180.0
1,store_2,2,0,28/10/2010,NaN,Culture & Media & Technology,Electronics and household appliances,Monobrand electronics,2954560.0,326322.0,...,13.0,5621.56,332009.67,204.01,12048.74,50594,2988087,248,504.829652,240.0
2,store_3,3,0,28/10/2010,NaN,Bags & Footwear & Accessories,Accessories,"Mixed: small accessories, bags and fancy jewel...",9706951.0,1189964.0,...,13.0,12013.67,330280.78,432.49,11890.11,108123,2972527,250,349.092231,240.0
3,store_4,4,0,28/10/2010,NaN,Services,Photo services,Photo Development,4217850.0,145534.0,...,13.0,4175.00,284250.33,152.74,10399.40,37575,2558253,246,345.275969,240.0
4,store_5,5,0,23/10/1990,NaN,Health & Beauty,"Perfumery, beautycare",Monobrand mainstream,1645623.0,503069.0,...,13.0,25665.44,294645.11,923.96,10607.22,230989,2651806,250,378.828395,300.0


In [158]:
merged.columns

Index(['store_name', 'store_id', 'mall_id', 'store_year_opened',
       'store_year_closed', 'store_branch_level_1', 'store_branch_level_2',
       'store_branch_level_3', 'store_sales_r12m', 'store_rent_r12m',
       'store_area', 'block_id', 'external_mall_id',
       'store_visits_monthly_average', 'store_window_flow_monthly_average',
       'store_visits_daily_average', 'store_window_flow_daily_average',
       'store_total_visits', 'store_total_window_flow',
       'days_with_traffic_data', 'average_dwell_time', 'median_dwell_time'],
      dtype='object')